In [292]:
%matplotlib inline
import pandas as pd
import glob
import os
import numpy as np
import json
from bokeh.plotting import show, output_notebook, figure, ColumnDataSource
from bokeh.charts import Scatter
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.models import BoxZoomTool
from lightjob.cli import load_db
from lightjob.db import SUCCESS

def moving(l):
    v = 0.
    lm = []
    for val in l:
        v = v * 0.9999 + val * 0.0001
        lm.append(v)
    return lm
output_notebook()

def get_df():
    db = load_db('../students/.lightjob')
    rows = []
    jobs = db.jobs_with(state=SUCCESS)
    for job in jobs:
        folder = '../students/jobs/{}'.format(job['summary'])
        stats = os.path.join(folder, 'stats.csv')
        if not os.path.exists(stats):
            continue
        df_stats = pd.read_csv(stats)
        df_valid = pd.read_csv(os.path.join(folder, 'valid.csv'))
        df_valid = df_valid[df_valid.columns[0]].values
        hypers = job['content']['hypers']
        start_time = job['life'][0]['dt']
        end_time = job['life'][0]['dt']    
        acc = moving(df_stats['acc'])
        row = job['content'].copy()
        del row['hypers']
        row.update(hypers)

        row['train_acc'] = acc[::1250]
        row['valid_acc'] = df_valid.tolist()
        row['max_valid_acc'] = float(df_valid.max())
        row['last_train_acc'] = row['train_acc'][-1]
        row['max_train_acc'] = np.max(row['train_acc'])
        row['id'] = os.path.basename(folder)
        row['nb_updates'] = len(df_stats)
        row['start_time'] = start_time
        row['end_time'] = end_time
        row['n_epochs'] = (len(df_stats)*32)/40000
        rows.append(row)
        
    df = pd.DataFrame(rows)
    return df


Loading BokehJS ...

In [293]:
df = get_df()

In [294]:
df.sort_values(by='max_valid_acc', ascending=False)

,algo,data_source,end_time,fc,fc1,fc2,id,last_train_acc,lr,max_train_acc,...,n_epochs,nb_updates,nbf,nbf1,nbf2,sf,start_time,train_acc,valid_acc,xavier
373,adam,"dataset_simple,aux2",2017-04-17T23:57:35.826072,900.0,NaN,NaN,91e2c05e100ae416e69b7e983c5be906,0.923240,0.000172,0.927569,...,636.0,795000,NaN,96.0,256.0,5,2017-04-17T23:57:35.826072,"[1.25e-05, 0.0503498653549, 0.134135639655, 0....","[0.5365415335463258, 0.5955471246006391, 0.614...",True
382,adam,"dataset_simple,aux3",2017-04-18T00:06:24.214908,900.0,NaN,NaN,99dadfc9126baa825e47cb6466568164,0.953230,0.000172,0.956588,...,712.0,890000,NaN,96.0,256.0,5,2017-04-18T00:06:24.214908,"[1.25e-05, 0.0503214381186, 0.124248918534, 0....","[0.5653953674121406, 0.6261980830670927, 0.641...",True
385,adam,"dataset_simple,aux3",2017-04-18T00:06:24.266508,300.0,NaN,NaN,7c933b4b4e041d28f9b01b18957951e0,0.928189,0.000376,0.934320,...,498.0,622500,NaN,256.0,192.0,5,2017-04-18T00:06:24.266508,"[6.25e-06, 0.0501085695132, 0.126007018054, 0....","[0.5712859424920128, 0.650758785942492, 0.6535...",True
386,nesterov,"dataset_simple,aux3",2017-04-18T00:06:24.297265,900.0,NaN,NaN,321adde77ed430eab72e13fbfedb3586,0.932267,0.000636,0.937436,...,582.0,727500,NaN,128.0,128.0,3,2017-04-18T00:06:24.297265,"[3.125e-06, 0.0488462027619, 0.119336264444, 0...","[0.5641972843450479, 0.6188099041533547, 0.654...",True
383,nesterov,"dataset_simple,aux3",2017-04-18T00:06:24.230438,400.0,NaN,NaN,dd77669bb8de1599b1600c665bed70e3,0.930319,0.000440,0.935585,...,534.0,667500,NaN,192.0,192.0,3,2017-04-18T00:06:24.230438,"[1.25e-05, 0.04601953595, 0.113831645348, 0.16...","[0.5435303514376997, 0.6092252396166135, 0.638...",True
377,nesterov,"dataset_simple,aux2",2017-04-17T23:57:35.907173,900.0,NaN,NaN,005ad63a33c5df15f1cfff3c139ac4cc,0.901830,0.000636,0.906530,...,660.0,825000,NaN,128.0,128.0,3,2017-04-17T23:57:35.907173,"[3.125e-06, 0.0488784788399, 0.128698853975, 0...","[0.5177715654952076, 0.5834664536741214, 0.609...",True
379,adam,"dataset_simple,aux2",2017-04-17T23:57:35.954632,600.0,NaN,NaN,bbde7e162f9d1af38b5f473e137c5aaf,0.898622,0.000104,0.904271,...,726.0,907500,NaN,96.0,128.0,3,2017-04-17T23:57:35.954632,"[1.25e-05, 0.0437130243714, 0.116835801783, 0....","[0.4794329073482428, 0.5468250798722045, 0.575...",True
384,sgd,"dataset_simple,aux3",2017-04-18T00:06:24.246289,300.0,NaN,NaN,8f0bd4e8e915ead7dde260a456a6f738,0.924729,0.003494,0.930484,...,538.0,672500,NaN,128.0,256.0,5,2017-04-18T00:06:24.246289,"[1.5625e-05, 0.0301017759922, 0.0893779288479,...","[0.4946086261980831, 0.5909544728434505, 0.638...",True
376,adam,"dataset_simple,aux2",2017-04-17T23:57:35.891575,300.0,NaN,NaN,cda883556f01d869af1d6d6e4c41c094,0.893516,0.000376,0.896802,...,496.0,620000,NaN,256.0,192.0,5,2017-04-17T23:57:35.891575,"[6.25e-06, 0.0499633548429, 0.134054711515, 0....","[0.5352436102236422, 0.6181110223642172, 0.626...",True
375,sgd,"dataset_simple,aux2",2017-04-17T23:57:35.856572,300.0,NaN,NaN,dcb30369a52d4a823061f84a4f9c1418,0.891670,0.003494,0.895441,...,644.0,805000,NaN,128.0,256.0,5,2017-04-17T23:57:35.856572,"[1.5625e-05, 0.021040985574, 0.0804155030344, ...","[0.4293130990415335, 0.5354432907348243, 0.572...",True


In [295]:
hover = HoverTool(
    tooltips=[
        ('valid_acc', '@max_valid_acc'),
        ('train_acc', '@last_train_acc')
    ]
)
p = figure(tools=[hover])
p.circle('fc', 'max_valid_acc', source=ColumnDataSource(df))
show(p)

In [296]:
from bokeh.charts import Bar, output_file, show
p = Bar(df, 'data_source', values='max_valid_acc', agg='median')
p.legend.location = (0,0)
show(p)

In [297]:
best = df.sort_values(by='max_valid_acc', ascending=False).iloc[0].to_dict()
print(best['data_source'])
p = figure(tools=[HoverTool(), BoxZoomTool()])
e = np.arange(len(best['train_acc']))
p.line(e, best['train_acc'], color='blue', legend='train acc')
p.line(e, best['valid_acc'], color='orange', legend='valid acc')
p.legend.location = 'bottom_right'
show(p)

dataset_simple,aux2
